In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

In [33]:
import pandas as pd
import numpy as np

In [6]:
url="https://www.nickcave.com/lyrics/"
soup=BeautifulSoup(requests.get(url).text)

print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]><html class="no-js ie6 oldie" lang=en> <![endif]-->
<!--[if IE 7]><html class="no-js ie7 oldie" lang=en> <![endif]-->
<!--[if IE 8]><html class="no-js ie8 oldie" lang=en> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-US">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0, user-scalable=1" name="viewport"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <script>
   var gform;gform||(document.addEventListener("gform_main_scripts_loaded",function(){gform.scriptsLoaded=!0}),window.addEventListener("DOMContentLoaded",function(){gform.domLoaded=!0}),gform={domLoaded:!1,scriptsLoaded:!1,initializeOnLoaded:function(o){gform.domLoaded&&gform.scriptsLoaded?o():!gform.domLoaded&&gform.scriptsLoaded?window.addEventListener("DOMContentLoaded",o):document.addEventListener("gform_main_scripts_

In [28]:
table=[]
for album in soup.find_all('div',class_='lyric-album-list'):
    for song in album.find_all('a'):
        entry={}
        entry['Album']=album.find('h3').get_text()
        entry['Song']=song.get_text()
        table.append(entry)
df=pd.DataFrame(table)

In [30]:
def nopunc(string):
    punc_rem='''!()[]{};:'"\,<>.?@#$%^&*_~’'''
    punc_hyph='/' #songs with / in the title such as Weird Fishes/Arpeggi do not have the / removed but replaced by a hyphen
    for char in punc_rem:
        string=string.replace(char,'')
    for char in punc_hyph:
        string=string.replace(char,'-')
    return string

def generate_genius_url(artist,song):
    base_url='https://genius.com'
    artist=nopunc(artist)
    artist_hyphenated='-'.join(artist.split())
    song=nopunc(song)
    song_hyphenated='-'.join(song.split())
    ending='lyrics'
    url=base_url + '/' +'-'.join([artist_hyphenated,song_hyphenated,ending])
    return url

def multiline_verse_marker_removal(text):
    txt_array=text.splitlines()
    ind=[]
    i=True
    for line in txt_array:
        if line[0]=='[':
            i=False
        ind.append(i)
        if line[-1]==']':
            i=True
    result='\n'.join(np.array(txt_array)[ind])
    return result

def get_lyrics(artist, song,remove_verse_markers=True,print_=False):
    url=generate_genius_url(artist,song)
    soup=BeautifulSoup(requests.get(url).text,'html.parser')
    Genius_error_text='Oops! Page not found'
    if soup.get_text().find(Genius_error_text)>0:
        return None
    else:
        Genius_class_val='Lyrics__Container-sc-1ynbvzw-5 Dzxov' #from inspeciton. If code breaks in future, check here
        lyrics=soup.find_all('div', class_=Genius_class_val)
        lyrics_all="\n".join([section.get_text("\n") for section in lyrics])
        if remove_verse_markers:
            #lyrics_all=re.sub('\n?\[.*\]\n?','',lyrics_all)
            lyrics_all=multiline_verse_marker_removal(lyrics_all)
        if print_:
            print(lyrics_all)
        return lyrics_all

In [34]:
for song in df.Song:
    df['Lyrics']=get_lyrics('Nick Cave and the Bad Seeds',song)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Eine vorhandene Verbindung wurde vom Remotehost geschlossen', None, 10054, None))

In [ ]:
df